In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks')
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mounted at /content/drive
Sun Mar  3 12:12:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

In [ ]:
pip install tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import numpy as np
import sys
import tsfresh as tsf
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
df_outf = pd.read_csv('combined_demand_df_1h.csv')
df_outf = df_outf.drop('time_slot', axis=1)

df_inf = pd.read_csv('combined_supply_df_1h.csv')
df_inf = df_inf.drop('time_slot', axis=1)

if(df_outf.shape[0] != df_inf.shape[0]): # check if dem and inf are same length
    print("outflow and inflow are NOT of same length")
    sys.exit()

In [ ]:
df_outf

,4,12,13,24,41,42,43,45,48,50,...,237,238,239,243,244,246,249,261,262,263
0,3.0,0.0,3.0,7.0,5.0,2.0,22.0,4.0,81.0,7.0,...,35.0,39.0,28.0,0.0,2.0,9.0,20.0,2.0,15.0,54.0
1,1.0,0.0,2.0,4.0,22.0,14.0,6.0,0.0,72.0,20.0,...,16.0,27.0,44.0,1.0,3.0,12.0,32.0,6.0,18.0,49.0
2,2.0,0.0,2.0,2.0,11.0,9.0,5.0,0.0,63.0,14.0,...,17.0,23.0,21.0,0.0,3.0,13.0,17.0,5.0,7.0,27.0
3,3.0,0.0,0.0,2.0,3.0,7.0,2.0,1.0,31.0,5.0,...,7.0,10.0,8.0,0.0,1.0,3.0,10.0,1.0,2.0,12.0
4,0.0,0.0,0.0,2.0,8.0,4.0,0.0,0.0,20.0,8.0,...,2.0,4.0,6.0,0.0,2.0,4.0,3.0,2.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24808,2.0,0.0,37.0,6.0,27.0,4.0,51.0,12.0,140.0,26.0,...,339.0,179.0,199.0,1.0,0.0,105.0,147.0,23.0,80.0,126.0
24809,5.0,0.0,37.0,10.0,12.0,1.0,88.0,7.0,128.0,19.0,...,394.0,119.0,191.0,1.0,1.0,93.0,159.0,26.0,67.0,112.0
24810,3.0,0.0,27.0,10.0,4.0,4.0,49.0,7.0,161.0,26.0,...,281.0,76.0,176.0,0.0,0.0,88.0,184.0,16.0,35.0,99.0
24811,9.0,0.0,14.0,9.0,12.0,4.0,37.0,11.0,175.0,24.0,...,183.0,36.0,94.0,0.0,0.0,53.0,190.0,12.0,21.0,60.0


In [ ]:
zone_significant_lags = {'4': [1, 167, 142, 22], '12': [1, 21, 24, 4], '13': [1, 23, 17, 5], '24': [1, 23, 168, 143], '41': [1, 23, 168, 144], '42': [1, 23, 168, 144], '43': [1, 4, 25, 21], '45': [1, 23, 167, 19], '48': [1, 25, 22, 18], '50': [1, 23, 19, 167], '68': [1, 19, 23, 4], '74': [1, 23, 168, 20], '75': [1, 5, 23, 14], '79': [1, 142, 166, 25], '87': [1, 18, 23, 5], '88': [1, 5, 23, 18], '90': [1, 19, 25, 22], '100': [1, 23, 167, 143], '107': [1, 25, 19, 22], '113': [1, 18, 4, 23], '114': [1, 167, 22, 142], '116': [1, 23, 168, 144], '120': [1, 82, 78, 98], '125': [1, 23, 19, 143], '127': [1, 24, 168, 144], '128': [24, 2, 144, 72], '137': [1, 23, 5, 26], '140': [1, 25, 22, 8], '141': [1, 23, 19, 5], '142': [1, 23, 19, 5], '143': [1, 23, 5, 167], '144': [1, 23, 20, 167], '148': [1, 166, 141, 21], '151': [1, 23, 5, 168], '152': [1, 23, 168, 144], '153': [168, 23, 2, 26], '158': [1, 167, 22, 142], '161': [1, 25, 4, 169], '162': [1, 25, 22, 17], '163': [1, 5, 25, 19], '164': [1, 18, 5, 23], '166': [1, 23, 5, 20], '170': [1, 25, 19, 22], '186': [1, 23, 20, 168], '194': [1, 24, 5, 21], '202': [1, 23, 168, 26], '209': [1, 23, 20, 167], '211': [1, 23, 167, 143], '224': [1, 24, 21, 168], '229': [1, 23, 18, 11], '230': [1, 23, 18, 169], '231': [1, 23, 19, 5], '232': [1, 22, 167, 142], '233': [1, 23, 19, 5], '234': [1, 4, 18, 25], '236': [1, 25, 4, 13], '237': [1, 25, 4, 17], '238': [1, 23, 4, 20], '239': [1, 4, 23, 19], '243': [1, 24, 21, 168], '244': [1, 24, 16, 168], '246': [1, 25, 22, 19], '249': [1, 25, 21, 142], '261': [1, 4, 20, 23], '262': [1, 23, 20, 167], '263': [1, 23, 19, 167]}

In [ ]:
cluster_zones = {
    '238': ['113', '137', '153', '164', '166', '194', '236', '237', '238'],
    '50': ['50', '144', '186', '209', '224', '230', '243', '244', '262', '263'],
    '120': ['4', '114', '120', '148', '158'],
    '232': ['24', '41', '42', '100', '116', '127', '128', '152', '211', '232'],
    '141': ['12', '13', '68', '87', '140', '141', '142', '229', '231', '233'],
    '151': ['143', '151', '161'],
    '246': ['48', '90', '107', '162', '170', '246'],
    '239': ['43', '75', '88', '163', '234', '239', '261'],
    '74': ['45', '74', '79', '202'],
    '125': ['125', '249']}

In [ ]:
cluster_aggregated_ts_outflow = pd.DataFrame(columns=sorted(['238', '50', '120', '232', '141', '151', '246', '239', '74', '125'], key=lambda x: int(x)))
cluster_aggregated_ts_inflow = pd.DataFrame(columns=sorted(['238', '50', '120', '232', '141', '151', '246', '239', '74', '125'], key=lambda x: int(x)))

for cluster, zones in cluster_zones.items():
  cumulated_outflow_df = pd.DataFrame({cluster: np.zeros(24813)})
  cumulated_inflow_df = pd.DataFrame({cluster: np.zeros(24813)})
  for zone in zones:
    df_out = df_outf[[str(zone)]]
    cumulated_outflow_df[cluster] += df_out[zone]
    df_in = df_inf[[str(zone)]]
    cumulated_inflow_df[cluster] += df_in[zone]
  cumulated_outflow_df.reset_index(drop=True, inplace=True)
  cumulated_inflow_df.reset_index(drop=True, inplace=True)
  cluster_aggregated_ts_outflow[cluster] = cumulated_outflow_df[cluster]
  cluster_aggregated_ts_inflow[cluster] = cumulated_inflow_df[cluster]

In [ ]:
cluster_aggregated_ts_outflow

,50,74,120,125,141,151,232,238,239,246
0,166.0,44.0,25.0,24.0,173.0,74.0,59.0,183.0,113.0,182.0
1,180.0,73.0,28.0,41.0,212.0,68.0,102.0,160.0,111.0,197.0
2,122.0,48.0,30.0,25.0,132.0,42.0,57.0,114.0,84.0,175.0
3,58.0,28.0,22.0,12.0,76.0,17.0,35.0,40.0,33.0,82.0
4,40.0,18.0,3.0,3.0,31.0,7.0,24.0,20.0,22.0,47.0
...,...,...,...,...,...,...,...,...,...,...
24808,623.0,125.0,110.0,173.0,1092.0,528.0,145.0,1258.0,718.0,937.0
24809,590.0,138.0,132.0,181.0,957.0,435.0,112.0,1105.0,711.0,884.0
24810,599.0,171.0,191.0,221.0,891.0,396.0,110.0,820.0,669.0,843.0
24811,501.0,218.0,209.0,217.0,692.0,275.0,143.0,619.0,576.0,735.0


In [ ]:
cluster_aggregated_ts_inflow

,50,74,120,125,141,151,232,238,239,246
0,117.0,48.0,23.0,14.0,200.0,47.0,74.0,173.0,125.0,123.0
1,158.0,74.0,38.0,28.0,196.0,48.0,103.0,128.0,119.0,113.0
2,120.0,49.0,33.0,15.0,140.0,31.0,64.0,92.0,68.0,114.0
3,63.0,26.0,11.0,10.0,58.0,12.0,41.0,39.0,38.0,57.0
4,24.0,12.0,6.0,7.0,21.0,9.0,30.0,27.0,13.0,31.0
...,...,...,...,...,...,...,...,...,...,...
24808,678.0,210.0,232.0,187.0,1171.0,338.0,260.0,1216.0,685.0,797.0
24809,698.0,193.0,190.0,113.0,1058.0,318.0,249.0,1048.0,681.0,766.0
24810,664.0,202.0,210.0,106.0,974.0,300.0,228.0,895.0,590.0,764.0
24811,573.0,185.0,209.0,114.0,811.0,220.0,219.0,720.0,422.0,673.0


In [ ]:
cluster_significant_lags = {}

for cluster in cluster_zones.keys():
  cluster_significant_lags[cluster] = zone_significant_lags[cluster]

print(cluster_significant_lags)

{'238': [1, 23, 4, 20], '50': [1, 23, 19, 167], '120': [1, 82, 78, 98], '232': [1, 22, 167, 142], '141': [1, 23, 19, 5], '151': [1, 23, 5, 168], '246': [1, 25, 22, 19], '239': [1, 4, 23, 19], '74': [1, 23, 168, 20], '125': [1, 23, 19, 143]}


In [ ]:
sorted_cluster_significant_lags = {k: cluster_significant_lags[k] for k in sorted(cluster_significant_lags, key=lambda x: int(x))}

print(sorted_cluster_significant_lags)

{'50': [1, 23, 19, 167], '74': [1, 23, 168, 20], '120': [1, 82, 78, 98], '125': [1, 23, 19, 143], '141': [1, 23, 19, 5], '151': [1, 23, 5, 168], '232': [1, 22, 167, 142], '238': [1, 23, 4, 20], '239': [1, 4, 23, 19], '246': [1, 25, 22, 19]}


In [ ]:
class create_raw_features(object):
    #def __init__(self,):

    def create_lag_features(ts,lags_list):
        lag_ts0 = ts.shift(periods = lags_list[0])
        lag_ts1 = ts.shift(periods = lags_list[1])
        lag_ts2 = ts.shift(periods = lags_list[2])
        lag_ts3 = ts.shift(periods = lags_list[3])
        return lag_ts0,lag_ts1,lag_ts2,lag_ts3

    def create_diff_ts_features(ts,diff_list):
        diff1_p1 = ts.diff(periods=diff_list[0])
        diff1_p2 = ts.diff(periods=diff_list[1])

        diff2_p1 = ts.diff(periods=diff_list[0]).diff()
        return diff1_p1,diff1_p2,diff2_p1

In [ ]:
dt = pd.Timestamp("2021-01-01") + pd.to_timedelta(range(df_inf.shape[0]), "H")
raw_feat_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag_0','inflow_lag_1','inflow_lag_2','inflow_lag_3','outflow_lag_0','outflow_lag_1','outflow_lag_2','outflow_lag_3'])

diff_list = [1,2]

In [ ]:
"""
stack all the raw feature time series of clusters vertically

tsfresh features are then added to these raw features to enlarge the feature set
"""

# inflow means supply, outflow means demand

for c in cluster_aggregated_ts_inflow.columns:
    lags_list = sorted_cluster_significant_lags[c]
    temp_df = pd.DataFrame(columns=['time_idx','datetime','group','target','inflow_lag_0','inflow_lag_1','inflow_lag_2','inflow_lag_3','outflow_lag_0','outflow_lag_1','outflow_lag_2','outflow_lag_3', 'inflow', "inflow_diff1_per1", 'inflow_diff1_per2', 'inflow_diff2_per1'])
    temp_df['time_idx'] = range(cluster_aggregated_ts_inflow.shape[0])
    temp_df['datetime'] = dt
    temp_df['target'] = cluster_aggregated_ts_outflow[c]
    temp_df['group'] = np.repeat(int(c),cluster_aggregated_ts_inflow.shape[0])
    temp_df['inflow_lag_0'],temp_df['inflow_lag_1'],temp_df['inflow_lag_2'],temp_df['inflow_lag_3'] = create_raw_features.create_lag_features(cluster_aggregated_ts_inflow[c],lags_list) # <-- do u need to change lags_list to supply ones -> no dont need
    temp_df['outflow_lag_0'],temp_df['outflow_lag_1'],temp_df['outflow_lag_2'],temp_df['outflow_lag_3'] = create_raw_features.create_lag_features(cluster_aggregated_ts_outflow[c],lags_list)

    temp_df['inflow'] = cluster_aggregated_ts_inflow[c]
    temp_df['inflow_diff1_per1'],temp_df['inflow_diff1_per2'], temp_df['inflow_diff2_per1'] = create_raw_features.create_diff_ts_features(cluster_aggregated_ts_inflow[c],diff_list)

    raw_feat_df = pd.concat([raw_feat_df, temp_df],axis=0)

raw_feat_df = raw_feat_df.dropna().reset_index(drop=True)
raw_feat_df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,outflow_lag_2,outflow_lag_3,inflow,inflow_diff1_per1,inflow_diff1_per2,inflow_diff2_per1
0,167,2021-01-07 23:00:00,50,59.0,107.0,38.0,12.0,117.0,122.0,41.0,27.0,166.0,67.0,-40.0,-77.0,-3.0
1,168,2021-01-08 00:00:00,50,32.0,67.0,14.0,30.0,158.0,59.0,23.0,45.0,180.0,34.0,-33.0,-73.0,7.0
2,169,2021-01-08 01:00:00,50,32.0,34.0,17.0,84.0,120.0,32.0,23.0,236.0,122.0,24.0,-10.0,-43.0,23.0
3,170,2021-01-08 02:00:00,50,24.0,24.0,4.0,162.0,63.0,32.0,8.0,328.0,58.0,12.0,-12.0,-22.0,-2.0
4,171,2021-01-08 03:00:00,50,9.0,12.0,12.0,243.0,24.0,24.0,27.0,392.0,40.0,6.0,-6.0,-18.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247120,24808,2023-10-31 16:00:00,246,937.0,833.0,866.0,701.0,161.0,1060.0,1083.0,791.0,161.0,797.0,-36.0,-30.0,-42.0
247121,24809,2023-10-31 17:00:00,246,884.0,797.0,842.0,560.0,75.0,937.0,979.0,537.0,101.0,766.0,-31.0,-67.0,5.0
247122,24810,2023-10-31 18:00:00,246,843.0,766.0,772.0,324.0,37.0,884.0,959.0,358.0,60.0,764.0,-2.0,-33.0,29.0
247123,24811,2023-10-31 19:00:00,246,735.0,764.0,701.0,161.0,32.0,843.0,791.0,161.0,55.0,673.0,-91.0,-93.0,-89.0


In [ ]:
"""
Create TSFRESH Time series features

"""

win_lengths = [24] # hyperparameter

# drop all categorical columns

col_list = raw_feat_df.columns

feature_settings = {'fft_coefficient': [{'coeff': 0, 'attr': 'real'}, {'coeff': 1, 'attr': 'real'}],
                    #'mean': None,
                    #'standard_deviation': None,
                    'skewness': None,
                    'kurtosis': None,
                    #'minimum': None,
                    #'maximum': None,
                    #'sum_values': None,
                    #'variance': None,
                    #'absolute_sum_of_changes': None,
                    'count_above_mean': None,
                    'count_below_mean': None,
                    #'longest_strike_above_mean': None,
                    #'longest_strike_below_mean': None,
                    #'number_crossing_m': [{'m': 0}],
                    'autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
                    'partial_autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}]}


for c in col_list[4:]: # select 1 numeric column at a time, leave out target and long lags
    print('time series name: ',c)
    for win in win_lengths:
        num_df = raw_feat_df.loc[:,['datetime','group',c]]
        print('num_df.shape', num_df.shape)
        df_rolled = tsf.utilities.dataframe_functions.roll_time_series(num_df,column_id='group',column_sort='datetime',max_timeshift=win)
        df_rolled.drop(columns=['group'], inplace=True)
        df_features = tsf.extract_features(df_rolled, column_id='id', column_sort='datetime',
                                            default_fc_parameters=feature_settings)
        del df_rolled # free-up the RAM

        # Remove any missing values
        imputed_features = impute(df_features)
        del df_features # free-up the RAM

        # Select a subset of relevant features using tsfresh's built-in feature selection function
        raw_feat_df.index= range(raw_feat_df.shape[0])
        imputed_features.index = range(raw_feat_df.shape[0])
        selected_features = tsf.select_features(imputed_features, raw_feat_df['target'])
        del imputed_features # free-up the RAM

        # Print the names of the selected features
        print('selected_features.columns',selected_features.columns)

        print('selected_features.columns: ',selected_features.columns)
        print('selected_features.shape: ',selected_features.shape)

        print('all_stns_df.columns: ',raw_feat_df.columns)
        print('all_stns_df.shape: ',raw_feat_df.shape)

        raw_feat_df = pd.concat([raw_feat_df,selected_features],axis=1)

        del num_df # free-up the RAM
        del selected_features # free-up the RAM

raw_feat_df.shape

time series name:  inflow_lag_0
num_df.shape (247125, 3)


Feature Extraction: 100%|██████████| 20/20 [02:11<00:00,  6.57s/it]


selected_features.columns Index(['inflow_lag_0__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_0__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_0__skewness', 'inflow_lag_0__count_above_mean',
       'inflow_lag_0__autocorrelation__lag_1',
       'inflow_lag_0__count_below_mean',
       'inflow_lag_0__autocorrelation__lag_2',
       'inflow_lag_0__autocorrelation__lag_3',
       'inflow_lag_0__partial_autocorrelation__lag_2',
       'inflow_lag_0__partial_autocorrelation__lag_1',
       'inflow_lag_0__partial_autocorrelation__lag_3',
       'inflow_lag_0__kurtosis'],
      dtype='object')
selected_features.columns:  Index(['inflow_lag_0__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_0__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_0__skewness', 'inflow_lag_0__count_above_mean',
       'inflow_lag_0__autocorrelation__lag_1',
       'inflow_lag_0__count_below_mean',
       'inflow_lag_0__autocorrelation__lag_2',
       'inflow_lag_0__auto

Feature Extraction: 100%|██████████| 20/20 [02:12<00:00,  6.60s/it]


selected_features.columns Index(['inflow_lag_1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_1__skewness', 'inflow_lag_1__count_above_mean',
       'inflow_lag_1__autocorrelation__lag_1',
       'inflow_lag_1__count_below_mean',
       'inflow_lag_1__autocorrelation__lag_2',
       'inflow_lag_1__autocorrelation__lag_3',
       'inflow_lag_1__partial_autocorrelation__lag_2',
       'inflow_lag_1__partial_autocorrelation__lag_1',
       'inflow_lag_1__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow_lag_1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_lag_1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_lag_1__skewness', 'inflow_lag_1__count_above_mean',
       'inflow_lag_1__autocorrelation__lag_1',
       'inflow_lag_1__count_below_mean',
       'inflow_lag_1__autocorrelation__lag_2',
       'inflow_lag_1__autocorrelation__lag_3',
       'infl

...

Feature Extraction: 100%|██████████| 20/20 [02:15<00:00,  6.78s/it]


selected_features.columns Index(['inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per2__skewness',
       'inflow_diff1_per2__autocorrelation__lag_2',
       'inflow_diff1_per2__autocorrelation__lag_1',
       'inflow_diff1_per2__autocorrelation__lag_3',
       'inflow_diff1_per2__partial_autocorrelation__lag_1',
       'inflow_diff1_per2__partial_autocorrelation__lag_2',
       'inflow_diff1_per2__count_below_mean',
       'inflow_diff1_per2__count_above_mean', 'inflow_diff1_per2__kurtosis',
       'inflow_diff1_per2__partial_autocorrelation__lag_3'],
      dtype='object')
selected_features.columns:  Index(['inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff1_per2__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff1_per2__skewness',
       'inflow_diff1_per2__autocorrelation__lag_2',
       'inflow_diff1_per2__autocorrelation__lag_1',
       'inflow_d

Feature Extraction: 100%|██████████| 20/20 [02:17<00:00,  6.88s/it]


selected_features.columns Index(['inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff2_per1__kurtosis',
       'inflow_diff2_per1__autocorrelation__lag_2',
       'inflow_diff2_per1__autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__lag_3',
       'inflow_diff2_per1__autocorrelation__lag_3',
       'inflow_diff2_per1__skewness', 'inflow_diff2_per1__count_above_mean'],
      dtype='object')
selected_features.columns:  Index(['inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_0',
       'inflow_diff2_per1__fft_coefficient__attr_"real"__coeff_1',
       'inflow_diff2_per1__kurtosis',
       'inflow_diff2_per1__autocorrelation__lag_2',
       'inflow_diff2_per1__autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__lag_1',
       'inflow_diff2_per1__partial_autocorrelation__lag_3',
  

(247125, 156)

In [ ]:
for c in raw_feat_df.columns:
    print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__count_above_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_0__kurtosis
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__count_above_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__partial_autocorrelation__lag

In [ ]:
raw_feat_df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0","inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_1",inflow_diff2_per1__kurtosis,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_3,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__skewness,inflow_diff2_per1__count_above_mean
0,167,2021-01-07 23:00:00,50,59.0,107.0,38.0,12.0,117.0,122.0,41.0,...,-3.0,-4.786351,0.561537,-0.128245,-0.269178,-0.269166,-0.110965,-0.026810,-0.468564,0.0
1,168,2021-01-08 00:00:00,50,32.0,67.0,14.0,30.0,158.0,59.0,23.0,...,4.0,-10.000000,0.561537,-0.128245,-1.000000,-0.269166,-0.110965,-0.026810,-0.468564,1.0
2,169,2021-01-08 01:00:00,50,32.0,34.0,17.0,84.0,120.0,32.0,23.0,...,27.0,-18.000000,0.561537,-1.465116,-0.017442,-0.269166,-0.110965,-0.026810,0.670284,1.0
3,170,2021-01-08 02:00:00,50,24.0,24.0,4.0,162.0,63.0,32.0,8.0,...,25.0,-26.000000,0.891341,-0.741231,-0.406555,-0.406555,-0.110965,0.702128,1.279001,2.0
4,171,2021-01-08 03:00:00,50,9.0,12.0,12.0,243.0,24.0,24.0,27.0,...,31.0,-15.972136,1.663525,-0.630481,-0.373850,-0.373850,-0.110965,0.432843,1.254480,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247120,24808,2023-10-31 16:00:00,246,937.0,833.0,866.0,701.0,161.0,1060.0,1083.0,...,18.0,-155.248275,0.388287,-0.356099,0.135241,0.135241,0.053212,-0.073461,-0.347820,12.0
247121,24809,2023-10-31 17:00:00,246,884.0,797.0,842.0,560.0,75.0,937.0,979.0,...,-7.0,-110.884152,0.401069,-0.358260,0.141407,0.141407,0.077524,-0.060138,-0.317217,12.0
247122,24810,2023-10-31 18:00:00,246,843.0,766.0,772.0,324.0,37.0,884.0,959.0,...,68.0,38.090968,0.503437,-0.384094,0.142216,0.142216,0.048863,-0.096132,-0.416259,13.0
247123,24811,2023-10-31 19:00:00,246,735.0,764.0,701.0,161.0,32.0,843.0,791.0,...,-20.0,24.437373,0.209153,-0.370867,0.126136,0.126136,0.053884,-0.069905,-0.314286,13.0


In [ ]:
pip install pyarrow

In [ ]:
raw_feat_df.to_parquet('combined_dd_ss_raw_feat_data_cluster_aggregation.parquet')

In [ ]:
import pandas as pd
import os
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks')
df = pd.read_parquet('combined_dd_ss_raw_feat_data_cluster_aggregation.parquet', engine='pyarrow')
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0","inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_1",inflow_diff2_per1__kurtosis,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_3,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__skewness,inflow_diff2_per1__count_above_mean
0,167,2021-01-07 23:00:00,50,59.0,107.0,38.0,12.0,117.0,122.0,41.0,...,-3.0,-4.786351,0.561537,-0.128245,-0.269178,-0.269166,-0.110965,-0.026810,-0.468564,0.0
1,168,2021-01-08 00:00:00,50,32.0,67.0,14.0,30.0,158.0,59.0,23.0,...,4.0,-10.000000,0.561537,-0.128245,-1.000000,-0.269166,-0.110965,-0.026810,-0.468564,1.0
2,169,2021-01-08 01:00:00,50,32.0,34.0,17.0,84.0,120.0,32.0,23.0,...,27.0,-18.000000,0.561537,-1.465116,-0.017442,-0.269166,-0.110965,-0.026810,0.670284,1.0
3,170,2021-01-08 02:00:00,50,24.0,24.0,4.0,162.0,63.0,32.0,8.0,...,25.0,-26.000000,0.891341,-0.741231,-0.406555,-0.406555,-0.110965,0.702128,1.279001,2.0
4,171,2021-01-08 03:00:00,50,9.0,12.0,12.0,243.0,24.0,24.0,27.0,...,31.0,-15.972136,1.663525,-0.630481,-0.373850,-0.373850,-0.110965,0.432843,1.254480,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247120,24808,2023-10-31 16:00:00,246,937.0,833.0,866.0,701.0,161.0,1060.0,1083.0,...,18.0,-155.248275,0.388287,-0.356099,0.135241,0.135241,0.053212,-0.073461,-0.347820,12.0
247121,24809,2023-10-31 17:00:00,246,884.0,797.0,842.0,560.0,75.0,937.0,979.0,...,-7.0,-110.884152,0.401069,-0.358260,0.141407,0.141407,0.077524,-0.060138,-0.317217,12.0
247122,24810,2023-10-31 18:00:00,246,843.0,766.0,772.0,324.0,37.0,884.0,959.0,...,68.0,38.090968,0.503437,-0.384094,0.142216,0.142216,0.048863,-0.096132,-0.416259,13.0
247123,24811,2023-10-31 19:00:00,246,735.0,764.0,701.0,161.0,32.0,843.0,791.0,...,-20.0,24.437373,0.209153,-0.370867,0.126136,0.126136,0.053884,-0.069905,-0.314286,13.0


In [ ]:
for c in df.columns:
  print(c)

time_idx
datetime
group
target
inflow_lag_0
inflow_lag_1
inflow_lag_2
inflow_lag_3
outflow_lag_0
outflow_lag_1
outflow_lag_2
outflow_lag_3
inflow
inflow_diff1_per1
inflow_diff1_per2
inflow_diff2_per1
inflow_lag_0__fft_coefficient__attr_"real"__coeff_0
inflow_lag_0__fft_coefficient__attr_"real"__coeff_1
inflow_lag_0__skewness
inflow_lag_0__count_above_mean
inflow_lag_0__autocorrelation__lag_1
inflow_lag_0__count_below_mean
inflow_lag_0__autocorrelation__lag_2
inflow_lag_0__autocorrelation__lag_3
inflow_lag_0__partial_autocorrelation__lag_2
inflow_lag_0__partial_autocorrelation__lag_1
inflow_lag_0__partial_autocorrelation__lag_3
inflow_lag_0__kurtosis
inflow_lag_1__fft_coefficient__attr_"real"__coeff_0
inflow_lag_1__fft_coefficient__attr_"real"__coeff_1
inflow_lag_1__skewness
inflow_lag_1__count_above_mean
inflow_lag_1__autocorrelation__lag_1
inflow_lag_1__count_below_mean
inflow_lag_1__autocorrelation__lag_2
inflow_lag_1__autocorrelation__lag_3
inflow_lag_1__partial_autocorrelation__lag